## Dependencies

In [1]:
import glob
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add, Average
from googleqa_utilityscript import *
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
MODULE_BASE_PATH = '/kaggle/input/bert-base-uncased-huggingface/bert-base-uncased'
MODULE_PATH = MODULE_BASE_PATH + '/bert-base-uncased-tf_model.h5'
VOCAB_PATH = MODULE_BASE_PATH + '/bert-base-uncased-vocab.txt'
CONF_PATH = MODULE_BASE_PATH + '/bert-base-uncased-config.json'

model_path_list = glob.glob('/kaggle/input/157-googleq-a-train-3fold-bert-base-unc-qa/' + '*1.h5')
model_path_list += glob.glob('/kaggle/input/157-googleq-a-train-3fold-bert-base-unc-qa/' + '*2.h5')
model_path_list += glob.glob('/kaggle/input/157-googleq-a-train-3fold-bert-base-unc-qa/' + '*3.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/157-googleq-a-train-3fold-bert-base-unc-qa/model_fold_1.h5
/kaggle/input/157-googleq-a-train-3fold-bert-base-unc-qa/model_fold_2.h5
/kaggle/input/157-googleq-a-train-3fold-bert-base-unc-qa/model_fold_3.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = ['question_title', 'answer']

# Model parameters

In [5]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512
MAX_LENGTH_TITLE = 30
MAX_LENGTH_QUESTION = 479
MAX_LENGTH_ANSWER = 479

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=False)

# Test features
X_test = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                               MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
        + compute_input_arays_2(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Model

In [7]:
module_config = BertConfig.from_pretrained(CONF_PATH)
module_config.output_hidden_states=False

def model_fn():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    module_model = TFBertModel.from_pretrained(MODULE_PATH, config=module_config)
    sequence_output_question, pooled_output_question = module_model([input_word_ids_question, input_masks_question, segment_ids_question])
    sequence_output_answer, pooled_output_answer = module_model([input_word_ids_answer, input_masks_answer, segment_ids_answer])
    
    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_question = Dropout(0.2)(x_question)
    output_question = Dense(N_CLASS_QUESTION, kernel_initializer='glorot_uniform', activation="sigmoid", name="outputquestion")(x_question)
    
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x_answer = Dropout(0.2)(x_answer)
    output_answer = Dense(N_CLASS_ANSWER, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_answer")(x_answer)
    
    output = Concatenate()([output_question, output_answer])

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)

    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.914206,0.567524,0.155777,0.571599,0.629457,0.601304,0.629007,0.628423,0.296343,...,0.838054,0.921756,0.613697,0.974160,0.981304,0.853114,0.050598,0.034550,0.854571,0.934164
1,46,0.890992,0.528047,0.007570,0.767966,0.785263,0.935347,0.556826,0.458365,0.070281,...,0.689886,0.916183,0.621324,0.961996,0.975171,0.874769,0.742347,0.133810,0.293543,0.887156
2,70,0.886881,0.547671,0.027869,0.724785,0.791511,0.891697,0.570200,0.447117,0.035257,...,0.756106,0.920026,0.616065,0.967364,0.972693,0.842806,0.142086,0.051760,0.792707,0.884802
3,132,0.887466,0.441218,0.008366,0.701378,0.743423,0.916900,0.547282,0.407771,0.078688,...,0.727008,0.961510,0.719636,0.974407,0.985397,0.930788,0.728131,0.163829,0.699043,0.910792
4,200,0.904825,0.515775,0.038761,0.852459,0.748397,0.896094,0.551525,0.561903,0.034329,...,0.666977,0.922279,0.680649,0.974220,0.974347,0.862920,0.138962,0.100299,0.755610,0.931580


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.884792,0.557249,0.032495,0.763485,0.795587,0.868963,0.547858,0.474852,0.132757,...,0.745395,0.939496,0.683586,0.966513,0.977283,0.885079,0.471120,0.128953,0.560632,0.906155
std,2812.670060,0.038416,0.113754,0.042047,0.089819,0.100207,0.112243,0.033865,0.081457,0.129314,...,0.081260,0.022017,0.044091,0.010217,0.009196,0.030987,0.279031,0.053220,0.253065,0.023451
min,39.000000,0.746449,0.311438,0.004519,0.433669,0.285778,0.263695,0.480203,0.329247,0.008508,...,0.550719,0.839890,0.549128,0.926415,0.917955,0.744056,0.012022,0.016057,0.051851,0.814524
25%,2572.000000,0.861921,0.465482,0.010101,0.711776,0.756167,0.857774,0.523415,0.412367,0.039964,...,0.682657,0.926540,0.653799,0.961157,0.973254,0.868840,0.203040,0.090983,0.363196,0.893006
50%,5093.000000,0.886787,0.547631,0.015611,0.768287,0.812613,0.906125,0.543156,0.448607,0.085188,...,0.742917,0.943249,0.682363,0.968058,0.979407,0.890853,0.497115,0.129214,0.579871,0.909388
75%,7482.000000,0.911361,0.636459,0.034576,0.827699,0.858718,0.935247,0.565860,0.528746,0.178617,...,0.809495,0.956089,0.709921,0.973762,0.983745,0.907440,0.721090,0.166630,0.774906,0.921243
max,9640.000000,0.963104,0.850962,0.297370,0.969585,0.959533,0.979640,0.671701,0.719435,0.664994,...,0.918736,0.975197,0.804677,0.984955,0.991196,0.942003,0.907473,0.292915,0.968101,0.953864
